<a href="https://colab.research.google.com/github/doremococo/python-bootcamp/blob/main/chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>